In [1]:
import json

with open('documents.json', 'r') as documents_file:
    documents_raw = json.load(documents_file)

documents = []

for course_dict in documents_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [2]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [3]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [4]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

In [5]:
from collections import defaultdict

In [6]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [7]:
len(hashes), len(documents)

(947, 948)

In [8]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [9]:
hashes['593f7569']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [10]:
import json

In [11]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [12]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [13]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [23]:
prompt_template.format(**documents[3])

'You emulate a student who\'s taking our course.\nFormulate 5 questions this student might ask based on a FAQ record. The record\nshould contain the answer to the questions, and the questions should be complete and not too short.\nIf possible, use as fewer words as possible from the record. \n\nThe record:\n\nsection: General course-related questions\nquestion: Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?\nanswer: You don\'t need it. You\'re accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.\n\nProvide the output in parsable JSON without using code blocks:\n\n["question1", "question2", ..., "question5"]'

In [14]:
from google import genai
from google.genai import types

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
genai_client = genai.Client()

In [16]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    contents = types.Content(
      role='user',
      parts=[types.Part.from_text(text=prompt)]
    )
    
    response = genai_client.models.generate_content(
        model="gemini-2.5-flash", contents=contents
    )
    return response.text

In [19]:
json_response = generate_questions(documents[3])
print(json_response)

["After completing the registration for the Data Engineering Bootcamp, should I expect to receive an acceptance or confirmation email?",
"Is it absolutely necessary to complete the registration process before I can begin accessing course materials or submitting assignments?",
"If registration isn't a strict requirement for participation, what is its actual role or benefit within the course structure?",
"Will my submissions for assignments be validated against any official enrollment list, or can I submit them independently?",
"Without receiving a formal confirmation, how can I be certain that I am officially enrolled and ready to fully participate in the Data Engineering Bootcamp?"]


In [20]:
json.loads(json_response)

['After completing the registration for the Data Engineering Bootcamp, should I expect to receive an acceptance or confirmation email?',
 'Is it absolutely necessary to complete the registration process before I can begin accessing course materials or submitting assignments?',
 "If registration isn't a strict requirement for participation, what is its actual role or benefit within the course structure?",
 'Will my submissions for assignments be validated against any official enrollment list, or can I submit them independently?',
 'Without receiving a formal confirmation, how can I be certain that I am officially enrolled and ready to fully participate in the Data Engineering Bootcamp?']

In [24]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
results = {}

In [26]:
limit = 1; # len(documents)
index = 0
for doc in tqdm(documents):
    if (index >= limit):
        break
    index += 1
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|                                                                                                     | 1/948 [00:05<1:33:01,  5.89s/it]


In [32]:
results

{'c02e79ef': '[\n"Could you please confirm the exact date and time the course officially kicks off, and what kind of event will mark its beginning?",\n"Is there a specific calendar I should subscribe to for all course events, and are there any device restrictions for accessing it?",\n"What is the mandatory registration process I need to complete before the course commences?",\n"How will I receive official announcements and critical updates throughout the course?",\n"Is there a dedicated platform or community where I can engage with other students and course staff?"\n]'}

In [34]:
!wget https://github.com/DataTalksClub/llm-zoomcamp/raw/refs/heads/main/03-evaluation/search_evaluation/results.bin

--2025-07-15 10:31:02--  https://github.com/DataTalksClub/llm-zoomcamp/raw/refs/heads/main/03-evaluation/search_evaluation/results.bin
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/03-evaluation/search_evaluation/results.bin [following]
--2025-07-15 10:31:03--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/03-evaluation/search_evaluation/results.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 403252 (394K) [application/octet-stream]
Saving to: ‘results.bin.1’

results.bin.1       100%[===================>] 393.80

In [33]:
import pickle

In [39]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [40]:
results['1f6520ca']

'["Where can I find the prerequisites for this course?", "How do I check the prerequisites for this course?", "Where are the course prerequisites listed?", "What are the requirements for joining this course?", "Where is the list of prerequisites for the course?"]'

In [48]:
parsed_results = {}

for doc_id, json_questions in results.items():
    parsed_results[doc_id] = json.loads(json_questions)

In [42]:
print(json_questions)

[
"How can I resolve the Docker error 'invalid mode: \\Program Files\\Git\\var\\lib\\postgresql\\data'?",
"What should I do if I encounter an invalid mode error in Docker on Windows?",
"What is the correct mounting path to use in Docker for PostgreSQL data on Windows?",
"Can you provide an example of a correct Docker mounting path for PostgreSQL data?",
"How do I correct the mounting path error in Docker for \\\Program Files\\Git\\var\\lib\\postgresql\\data'?"
]


In [43]:
json_questions=[
r"How can I resolve the Docker error 'invalid mode: \Program Files\Git\var\lib\postgresql\data'?",
"What should I do if I encounter an invalid mode error in Docker on Windows?",
"What is the correct mounting path to use in Docker for PostgreSQL data on Windows?",
"Can you provide an example of a correct Docker mounting path for PostgreSQL data?",
r"How do I correct the mounting path error in Docker for \Program Files\Git\var\lib\postgresql\data'?"
]

In [44]:
json.dumps(json_questions)

'["How can I resolve the Docker error \'invalid mode: \\\\Program Files\\\\Git\\\\var\\\\lib\\\\postgresql\\\\data\'?", "What should I do if I encounter an invalid mode error in Docker on Windows?", "What is the correct mounting path to use in Docker for PostgreSQL data on Windows?", "Can you provide an example of a correct Docker mounting path for PostgreSQL data?", "How do I correct the mounting path error in Docker for \\\\Program Files\\\\Git\\\\var\\\\lib\\\\postgresql\\\\data\'?"]'

In [45]:
doc_id

'58c9f99f'

In [46]:
results[doc_id] = json.dumps(json_questions)

In [50]:
doc_index = {d['id']: d for d in documents}

In [52]:
final_results = []

for doc_id, questions in parsed_results.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [54]:
final_results[:10]

[('When does the course begin?', 'data-engineering-zoomcamp', 'c02e79ef'),
 ('How can I get the course schedule?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('What is the link for course registration?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('How can I receive course announcements?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('Where do I join the Slack channel?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('Where can I find the prerequisites for this course?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('How do I check the prerequisites for this course?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('Where are the course prerequisites listed?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('What are the requirements for joining this course?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('Where is the list of prerequisites for the course?',
  'data-engineering-zoomcamp',
  '1f6520ca')]

In [55]:
import pandas as pd

In [56]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [57]:
df.to_csv('ground-truth-data.csv', index=False)

In [58]:
!head ground-truth-data.csv

question,course,document
When does the course begin?,data-engineering-zoomcamp,c02e79ef
How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
Where can I find the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
How do I check the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
Where are the course prerequisites listed?,data-engineering-zoomcamp,1f6520ca
What are the requirements for joining this course?,data-engineering-zoomcamp,1f6520ca
